In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime
import sys
from lib import sparkStructuredStreaming
import os
%matplotlib inline
from matplotlib import pyplot as plt

### Set-up to stream from Kafka topic + read and write from/to Elasticsearch

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5,org.elasticsearch:elasticsearch-spark-20_2.11:7.6.2 pyspark-shell'

In [3]:
#"127.0.0.1:9092" (local) //"10.0.0.8:9092" (BACC)
bootstrap = "127.0.0.1:9092"

In [4]:
spark = SparkSession \
            .builder \
            .appName("KafkaIEXStructuredStreaming") \
            .master("local[*]") \
            .getOrCreate()

sqlContext = SQLContext(spark)

## 1. Historical Data

#### Read historical data from yahoo finance, write into Elasticsearch

In [5]:
interval = "1d"
period = "5d"
symbol = "AAPL"

h = sparkStructuredStreaming.history()
#h.to_es(symbol,interval,period,sqlContext)

## 2. Backtesting

### 2.1 Momentum Strategy

In [8]:
symbol = "MSFT"
interval = "1d"
momentum = 20
start_capital = 10000
commission = 0.002

sparkStructuredStreaming.backtest().momentum(symbol, interval, momentum, spark, start_capital, commission)

#### 2.1.2 p/e ratio strategy

In [49]:
from yahoofinancials import YahooFinancials

symbol = "AAPL"
interval = "1d"
pe_data = h.from_es(symbol,interval,spark)

eps = YahooFinancials(symbol).get_earnings_per_share()
udf_pe_ratio = udf(lambda x : x/eps)

def pe_position_func(pe):
    if pe<10:
        return -1
    elif pe>15:
        return 1
    else:
        return 0
    
udf_pe_position = udf(lambda x : pe_position_func(x))

pe_ratio = pe_data.select("date","Close").withColumn("pe",udf_pe_ratio("Close"))
pe_position = pe_ratio.withColumn("position",udf_pe_position("pe"))
pe_position = pe_position.select("date","Close","position")
pe_position.show(10)

+-------------------+------+--------+
|               date| Close|position|
+-------------------+------+--------+
|2019-05-05 22:00:00|205.79|       1|
|2019-05-06 22:00:00|200.24|       1|
|2019-05-07 22:00:00|200.28|       1|
|2019-05-08 22:00:00|198.13|       1|
|2019-05-09 22:00:00|195.38|       1|
|2019-05-12 22:00:00|184.03|       0|
|2019-05-13 22:00:00|186.94|       0|
|2019-05-14 22:00:00|189.18|       0|
|2019-05-15 22:00:00|188.35|       0|
|2019-05-16 22:00:00|187.28|       0|
+-------------------+------+--------+
only showing top 10 rows



## 3. Trading Simulation / Performance Evaluation with realtime Streams (Spark Streaming)

### Stream real time quotes from Kafka topic

In [ ]:
#use this for elasticsearch, otherwise it won't recognize date field
get_datetime_kafka = udf(lambda x : datetime.datetime.fromtimestamp((x-7200000)/ 1000.0).strftime("%Y-%m-%d"'T'"%H:%M:%S"))

sss = sparkStructuredStreaming.kafka_spark_stream(bootstrap)

parsedDF = sss.stream_quotes(spark)       

selectDF_es = parsedDF \
        .select(explode(array("quote_data")))\
        .select("col.*",get_datetime_kafka("col.latestUpdate").cast("String").alias("date"))

## 4. Visualize results, either here with Plotly or write results into Elasticsearch -> Kibana

### Test functions

In [7]:
def time_chart(df,interval):
    # use df with "timestamp", "latestPrice", "Watermark"
    # get open, high, low prices for each time interval
    interval_values = df.groupBy(
        window(df.timestamp, interval))\
        .agg(max("latestPrice").alias("high"),\
            min("latestPrice").alias("low"),\
            min("timestamp").alias("open_time"))\
        .select("window.start","window.end","high","low","open_time")\
        .withWatermark("start", interval)
    
    # join to get opening price from opening time
    chart = interval_values.join(df,interval_values.open_time == df.timestamp, "left")\
        .drop("open_time","timestamp")\
        .withColumnRenamed("latestPrice","open")
        
    return chart

In [8]:
def moving_average(spark, df, update, interval):
    # simple moving average for the interval "interval"
    
    windowdf = df.select(window(df.timestamp, interval, update), df.latestPrice)
    
    windowdf.createOrReplaceTempView("windowdf_sql")
    
    sma = spark.sql("""SELECT windowdf_sql.window AS time, avg(windowdf_sql.latestPrice) AS average
                    FROM windowdf_sql
                    Group BY windowdf_sql.window
                    """)   
    return sma